# Applying ML to Predict NYC Real Estate Value & Investment Opportunity

*This notebook scrapes streeteasy.com for data on listings for sale in the Five Boroughs and applies ML to evaluate my predictive model*

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import json
import random
# import os
# from collections import OrderedDict
# from urlparse import urlparse

In [2]:
boroughs = ['manhattan','brooklyn','queens','bronx','staten-island']

In [3]:
def url_def(lst):
    lst_of_urls = []
    for item in lst:
        lst_of_urls.append('http://streeteasy.com/for-sale/'+str(item)+'/status:listed?refined_search=true')
    return lst_of_urls

In [4]:
urls = url_def(boroughs)
urls

['http://streeteasy.com/for-sale/manhattan/status:listed?refined_search=true',
 'http://streeteasy.com/for-sale/brooklyn/status:listed?refined_search=true',
 'http://streeteasy.com/for-sale/queens/status:listed?refined_search=true',
 'http://streeteasy.com/for-sale/bronx/status:listed?refined_search=true',
 'http://streeteasy.com/for-sale/staten-island/status:listed?refined_search=true']

In [21]:
# to visually inspect all generated urls...
# ?! this doesn't make sense! fix it!
'''
resultas = []
for url in urls:
    driver = webdriver.Chrome()
    try:
        driver.get(url)
        time.sleep(0.5)
    except:
        
    driver.close()
    
'''

### Feature Selection

What would be the most useful features to collect for this project?

- **Type of house, Location of house, Neighborhood, Number of rooms, number of baths**, availability of amenities in building (laundry, doorman, super)?, proximity to transit, proximity to waterfront, (with the Price of house as target).

Features in bold are available on streeteasy...

Q. ***Can any useful features be engineered from those available or retrieved from an alternate source?***

### Scraped features

The features available from Streeteasy.com are:

 - House type, Geo-location, House address, No. of beds, No. of baths, Square area of house, Neighborhood, Price
 
*What features can be derived from these? What additional insight will these derived features provide?*
*Can more useful features be retrieved from other sources to complement Streeteasy?*

In [59]:
listing_type = []
lat = []
lng = []
address = []
no_of_beds = []
no_of_baths = []
sq_area = []
n_hood = []
price = []

### *Thoughts*

Do the project requirements dictate the statistical method/algorithm used? Will these, in turn, determine whether categorical or continuous variables are required?

- *Linear Regression*
- *Logistic Regression*
- *Random forest*

**N.B. This requirement directly dictates the page_scrape function below.**

**I'm going with numerical variables where possible...**

In [41]:
def page_scrape(page):
    count = 0
    # time.sleep(5)
    listings = page.find_element_by_xpath('//*[@id="result-details"]/ul').find_elements_by_tag_name('li')
    # collect data here by iterating through each listing and appending to our lists
    for l in listings[:14]:
        # initiate a counter to help identify at what listing the code breaks, if it does...
        # REMINDER! Also, a counter for the number of pages scraped should be implemented in the function that navigates pages
        count +=1
        print count
        
        # longitude and latitude
        g = None
        try:
            g = l.get_attribute('se:map:point')
            if g:
                lt, ln = g.split(',')
                lat.append(float(lt))
                lng.append(float(ln))
            else:
                lat.append('N/A')
                lng.append('N/A')
        except:
            lat.append('N/A')
            lng.append('N/A')
        # time.sleep(1)
        
        # address
        ad = None
        try:
            ad = l.find_element_by_class_name('details-title').text.split('\n')[0]
            if ad:
                address.append(ad)
            else:
                address.append('N/A')
        except:
            address.append('N/A')
        # time.sleep(1)
        
        # price
        p = None
        try:
            p = float(l.find_element_by_class_name('price').text.replace('$','').replace(',', ''))
            if p:
                price.append(p)
            else:
                price.append('N/A')
        except:
            price.append('N/A')
        # time.sleep(1)
        
        # number of beds
        bd_detail = None
        try:
            bd_detail = l.find_element_by_class_name('details_info').find_element_by_tag_name('span')
            if bd_detail.text.find('bed') > 0:
                no_of_beds.append(float(bd_detail.text.split(' ')[0]))
            # do we want this as a string or float? what are the regression/ml requirements?
            else:
                no_of_beds.append('N/A')
        except:
            no_of_beds.append('N/A')
        # time.sleep(1)
        
        # number of baths
        baths = None
        try:
            lstn_details = l.find_element_by_class_name('details_info').find_elements_by_tag_name('span')
            for detail in lstn_details:
                if detail.text.find('bath') > 0:
                    try:
                        baths = float(detail.text.split(' ')[0])
                    except:
                        baths = 'N/A'
        except:
            baths = 'N/A'
        no_of_baths.append(baths)
        # time.sleep(1)
        
        # square area NB: value in previous listing is being appended to next listing. FIX!
        # update: fixed.
        area = None
        try:
            l_details = l.find_element_by_class_name('details_info').find_elements_by_tag_name('span')
            for detail in l_details:
                 if detail.text.find('ft') > 0:
                    area = float(detail.text.split(' ')[0].replace(',', ''))
            if area:
                sq_area.append(area)
            else:
                sq_area.append('N/A')
        except:
            sq_area.append('N/A')
        # time.sleep(1)
        
        # listing type and neighborhood
        l_type = None
        nhood = None
        try:
            area_details = l.find_elements_by_class_name('details_info')[1].text
            l_type, nhood = area_details.split(' in ')
            if l_type:
                listing_type.append(l_type)
                # a spell-checker is required to correct mispells in house type e.g. 'Condop' instead of 'Condo'
                # update: condop is a legit building type
            else:
                listing_type.append('N/A')
            if nhood:
                n_hood.append(nhood)
            else:
                n_hood.append('N/A')
        except:
            listing_type.append('N/A')
            n_hood.append('N/A')
        # time.sleep(1)
    if count == 14:
        print('Moving on to the next page...')
    # streeteasy introduces a captcha when they suspect scraping. How will this be overridden?
    return listings

In [58]:
def next_page():
    nxt = listns[-1].find_element_by_class_name('next')
    nxt.click()

In [61]:
driver = webdriver.Firefox()
driver.get(urls[0])
time.sleep(2)
'''
for i in range(0, len(urls)):
    driver.get(urls[i])
    time.sleep(2)
    # call the page_scraper routine/function to handle each borough
    '''
# how do we iterate over the many result pages?
t = time.time()
listns = page_scrape(driver)
last_page = int(listns[16].find_elements_by_class_name('page')[-1].text)
time.sleep(2.5)
counter = 1
print 'Counter:', counter
next_page()
for i in range(1, last_page):
    counter += 1
    # what's req.?
    delay = random.uniform(2.5, 5)
    listns = page_scrape(driver)
    next_page()
    print 'Counter:', counter
    time.sleep(delay)
total_time = time.time() - t
print total_time

1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 6
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 7
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 8
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 9
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 10
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 11
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 12
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 13
1
2
3
4
5
6
7
8
9
10
11
12
13
14
M

10
11
12
13
14
Moving on to the next page...
Counter: 110
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 111
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 112
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 113
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 114
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 115
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 116
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 117
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 118
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 119
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 120
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 121
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 122
1
2
3
4
5
6
7
8
9
10
11
12
13


3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 218
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 219
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 220
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 221
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 222
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 223
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 224
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 225
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 226
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 227
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 228
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 229
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 230
1
2
3
4
5
6
7
8


1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 326
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 327
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 328
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 329
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 330
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 331
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 332
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 333
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 334
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 335
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 336
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 337
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 338
1
2
3
4
5
6


Counter: 433
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 434
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 435
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 436
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 437
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 438
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 439
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 440
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 441
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 442
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 443
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 444
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 445
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Moving on to the next page...
Counter: 446

WebDriverException: Message: Reached error page: about:neterror?e=nssFailure2&u=https%3A//a1501.casalemedia.com/ifnotify%3Fc%3D2C564B%26r%3D6209C7DF%26t%3D595B3CA6%26u%3DV1ZzYjFjQW9KbUVBQUVqc2J3MEFBQUJx%26m%3D01b454f7faf31c132cc53c13a716f187%26wp%3D2%26aid%3D65685982EEC5B32A%26tid%3DE139%26s%3D2A5AA%26cp%3D0.02%26n%3Dstreeteasy.com%26pr%3Dxx%26epr%3DJOD133P1Q6xZOcoUz%252B967g&c=UTF-8&f=regular&d=The%20connection%20to%20a1501.casalemedia.com%20was%20interrupted%20while%20the%20page%20was%20loading.


In [63]:
counter

472

In [62]:
driver.close()

In [64]:
dic = {'building type':listing_type, 'latitude':lat, 'longitude':lng, 'address':address, 'beds':no_of_beds, 'baths':no_of_baths, 'area':sq_area, 'neighborhood':n_hood, 'price':price}
'''for key in dic:
    print key
    print dic[key]
    print len(dic[key])
    '''

beds
[1.0, 1.0, 2.0, 2.0, 1.0, 1.5, 1.0, 'N/A', 1.0, 2.0, 3.0, 1.0, 2.0, 'N/A', 4.0, 2.0, 1.0, 1.0, 2.0, 4.0, 3.0, 2.0, 'N/A', 1.0, 2.0, 2.0, 'N/A', 1.0, 5.0, 6.0, 5.0, 1.0, 2.0, 5.0, 1.0, 2.0, 1.0, 'N/A', 'N/A', 1.0, 1.0, 1.0, 1.0, 'N/A', 1.0, 2.0, 1.0, 2.0, 2.0, 'N/A', 2.0, 1.0, 5.0, 4.0, 2.0, 1.0, 4.0, 4.0, 3.0, 2.0, 3.0, 2.0, 1.0, 6.0, 1.0, 3.0, 6.0, 1.0, 3.0, 2.0, 3.0, 3.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 'N/A', 2.0, 'N/A', 1.0, 2.0, 3.0, 2.0, 2.0, 1.0, 3.0, 2.0, 2.0, 1.0, 'N/A', 2.0, 2.0, 1.0, 2.0, 1.0, 4.0, 1.0, 1.0, 2.0, 3.0, 3.0, 5.0, 6.0, 4.0, 'N/A', 'N/A', 2.0, 'N/A', 1.0, 2.0, 1.0, 4.0, 2.0, 3.0, 'N/A', 1.0, 3.0, 2.0, 3.0, 4.0, 3.0, 1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 1.0, 2.0, 1.0, 1.0, 3.0, 2.0, 1.0, 'N/A', 3.0, 'N/A', 1.0, 3.0, 1.0, 1.0, 1.0, 'N/A', 4.0, 1.0, 3.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0, 5.0, 2.0, 1.0, 1.5, 3.0, 3.0, 3.0, 1.0, 3.0, 2.0, 1.0, 3.0, 2.0, 3.0, 3.0, 1.0, 3.0, 2.0, 3.0, 3.0, 3.0, 3.0, 4.0, 2.0, 1.0, 2.0, 3.0, 3.0, 2.0, 3.0, 1.0

In [68]:
data = pd.DataFrame(dic)
# data.to_csv('Streeteasy_data.csv') "The data is not yet good enough to warrant saving to disk...
data.head()

,address,area,baths,beds,building type,latitude,longitude,neighborhood,price
0,527 West 27th Street #3B,758,1,1,Condo,40.751,-74.0035,West Chelsea,1.96e+06
1,420 East 55th Street #4G,850,1,1,Co-op,40.7561,-73.9626,Sutton Place,685000
2,352 West 110th Street #9C,1000,2,2,Condo,40.8008,-73.9602,Manhattan Valley,1.15e+06
3,1160 Third Avenue #15E,N/A,1,2,Co-op,40.7671,-73.963,Lenox Hill,849000
4,10 West End Avenue #4C,N/A,1,1,Condo,40.7717,-73.99,Lincoln Square,1.19e+06


In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6608 entries, 0 to 6607
Data columns (total 9 columns):
address          6608 non-null object
area             6608 non-null object
baths            6565 non-null object
beds             6608 non-null object
building type    6608 non-null object
latitude         6608 non-null object
longitude        6608 non-null object
neighborhood     6608 non-null object
price            6608 non-null object
dtypes: object(9)
memory usage: 464.7+ KB


In [76]:
print data.describe()

                   address  area   baths    beds building type     latitude  \
count                 6608  6608  6565.0  6608.0          6608  6608.000000   
unique                5786  1584    23.0    19.0             8  1601.000000   
top     30 Park Place #46A   N/A     1.0     2.0         Condo    40.710175   
freq                     7  2358  2762.0  2055.0          3242    44.000000   

          longitude     neighborhood      price  
count   6608.000000             6608     6608.0  
unique  1366.000000               55     1306.0  
top      -73.989319  Upper West Side  1995000.0  
freq      44.000000              579       65.0  


### Handling 'N/A' values and outliers...

*Are samples with missing data discarded or replaced with the feature median? What is the norm as pertains to this situation...?*

Are statistical outliers really outliers in this use case? (Yes/**No**)?

### Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# sns.set(style='whitegrid', context='notebook')


### Model Equations
- Multivariate Linear Regression:
 - $y = w_0x_0 + w_1x_1 + ... + w_mx_m = \sum\limits_{i=0}^{\infty} w_ix_i = w^Tx$

In [ ]:
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import metrics


In [7]:
import numpy as np
help(np.c_)

Help on CClass in module numpy.lib.index_tricks object:

class CClass(AxisConcatenator)
 |  Translates slice objects to concatenation along the second axis.
 |  
 |  This is short-hand for ``np.r_['-1,2,0', index expression]``, which is
 |  useful because of its common occurrence. In particular, arrays will be
 |  stacked along their last axis after being upgraded to at least 2-D with
 |  1's post-pended to the shape (column vectors made out of 1-D arrays).
 |  
 |  For detailed documentation, see `r_`.
 |  
 |  Examples
 |  --------
 |  >>> np.c_[np.array([[1,2,3]]), 0, 0, np.array([[4,5,6]])]
 |  array([[1, 2, 3, 0, 0, 4, 5, 6]])
 |  
 |  Method resolution order:
 |      CClass
 |      AxisConcatenator
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from AxisConcatenator:
 |  
 |  __getitem__(self, key)
 |  
 |  __getslice__(self, i, j)
 |  
 |  __le

### Performance Measure

RMSE